本次以英雄联盟对局胜负预测任务为基础，要求实现决策树算法相关细节，加深对算法的理解，并了解做机器学习任务的大致流程。

### 任务介绍
英雄联盟（League of Legends，LoL）是一个多人在线竞技游戏，由拳头游戏（Riot Games）公司出品。在游戏中，每位玩家控制一位有独特技能的英雄，红蓝两支队伍各有五位玩家进行对战，目标是摧毁对方的基地水晶。水晶有多座防御塔保护，通常需要先摧毁一些防御塔再摧毁水晶。玩家所控制的英雄起初非常弱，需要不断击杀小兵、野怪和对方英雄来获得金币、经验。经验可以提升英雄等级和技能等级，金币可以用来购买装备提升攻击、防御等属性。对战过程中一般没有己方单位在附近的地点是没有视野的，即无法看到对面单位，双方可以通过使用守卫来监视某个地点，洞察对面走向、制定战术。
本数据集来自[Kaggle](https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min)，包含了9879场钻一到大师段位的单双排对局，对局双方几乎是同一水平。每条数据是前10分钟的对局情况，每支队伍有19个特征，红蓝双方共38个特征。这些特征包括英雄击杀、死亡，金钱、经验、等级情况等等。一局游戏一般会持续30至40分钟，但是实际前10分钟的局面很大程度上影响了之后胜负的走向。作为最成功的电子竞技游戏之一，对局数据、选手数据的量化与研究具有重要意义，可以启发游戏将来的发展和改进。

本任务是希望同学们依据注释的要求，对代码中空缺部分进行填写，**完成决策树模型的详细实现**，根据已有的对局前10分钟特征信息，预测最后获胜方是蓝色方还是红色方，了解执行一个**机器学习任务的大致流程**，并**提交代码和实验报告**。第一次作业也是一个机器学习小实验的例子，之后的作业可能不再提供预处理等流程代码，由同学们自己设计实验完成代码编写。

### 导入工具包
pandas是数据分析和处理常用的工具包，非常适合处理行列表格数据。numpy是数学运算工具包，支持高效的矩阵、向量运算。sklearn是机器学习常用工具包，包括了一些已经实现好的简单模型和一些常用数据处理方法、评价指标等函数。

In [33]:
from collections import Counter
import pandas as pd # type: ignore # 数据处理
import numpy as np # 数学运算
from sklearn.model_selection import train_test_split, cross_validate # 划分数据集函数
from sklearn.metrics import accuracy_score # 准确率函数
from graphviz import Digraph
RANDOM_SEED = 2020 # 固定随机种子

### 读入数据
假设数据文件放在`./data/`目录下，标准的csv文件可以用pandas里的`read_csv()`函数直接读入。文件共有40列，38个特征（红蓝方各19），1个标签列（blueWins），和一个对局标号（gameId）。对局标号不是标签也不是特征，可以舍去。

In [34]:
csv_data = R"D:\Code\yufengjihua\ML\实验\基于决策树的英雄联盟游戏胜负预测\data\high_diamond_ranked_10min.csv" # 数据路径
data_df = pd.read_csv(csv_data, sep=',') # 读入csv文件为pandas的DataFrame
data_df = data_df.drop(columns='gameId') # 舍去对局标号列
data_df

,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,0,28,2,1,9,6,11,0,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,0,12,1,0,5,5,5,0,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,0,15,0,0,7,11,4,1,1,0,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,0,43,1,0,4,5,5,1,0,1,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,0,75,4,0,6,6,6,0,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9874,1,17,2,1,7,4,5,1,1,0,...,0,15246,6.8,16498,229,34,-2519,-2469,22.9,1524.6
9875,1,54,0,0,6,4,8,1,1,0,...,0,15456,7.0,18367,206,56,-782,-888,20.6,1545.6
9876,0,23,1,0,6,7,5,0,0,0,...,0,18319,7.4,19909,261,60,2416,1877,26.1,1831.9
9877,0,14,4,1,2,3,3,1,1,0,...,0,15298,7.2,18314,247,40,839,1085,24.7,1529.8


###  数据概览
对于一个机器学习问题，在拿到任务和数据后，首先需要观察数据的情况，比如我们可以通过`.iloc[0]`取出数据的第一行并输出。不难看出每个特征都存成了float64浮点数，该对局蓝色方开局10分钟有小优势。同时也可以发现有些特征列是重复冗余的，比如blueGoldDiff表示蓝色队金币优势，redGoldDiff表示红色方金币优势，这两个特征是完全对称的互为相反数。blueCSPerMin是蓝色方每分钟击杀小兵数，它乘10就是10分钟所有小兵击杀数blueTotalMinionsKilled。在之后的特征处理过程中可以考虑去除这些冗余特征。
另外，pandas有非常方便的`describe()`函数，可以直接通过DataFrame进行调用，可以展示每一列数据的一些统计信息，对数据分布情况有大致了解，比如blueKills蓝色方击杀英雄数在前十分钟的平均数是6.14、方差为2.93，中位数是6，百分之五十以上的对局中该特征在4-8之间，等等。

In [35]:
print(data_df.iloc[0]) # 输出第一行数据
data_df.describe() # 每列特征的简单统计信息

blueWins                            0.0
blueWardsPlaced                    28.0
blueWardsDestroyed                  2.0
blueFirstBlood                      1.0
blueKills                           9.0
blueDeaths                          6.0
blueAssists                        11.0
blueEliteMonsters                   0.0
blueDragons                         0.0
blueHeralds                         0.0
blueTowersDestroyed                 0.0
blueTotalGold                   17210.0
blueAvgLevel                        6.6
blueTotalExperience             17039.0
blueTotalMinionsKilled            195.0
blueTotalJungleMinionsKilled       36.0
blueGoldDiff                      643.0
blueExperienceDiff                 -8.0
blueCSPerMin                       19.5
blueGoldPerMin                   1721.0
redWardsPlaced                     15.0
redWardsDestroyed                   6.0
redFirstBlood                       0.0
redKills                            6.0
redDeaths                           9.0


,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
count,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,...,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000
mean,0.499038,22.288288,2.824881,0.504808,6.183925,6.137666,6.645106,0.549954,0.361980,0.187974,...,0.043021,16489.041401,6.925316,17961.730438,217.349226,51.313088,-14.414111,33.620306,21.734923,1648.904140
std,0.500024,18.019177,2.174998,0.500002,3.011028,2.933818,4.064520,0.625527,0.480597,0.390712,...,0.216900,1490.888406,0.305311,1198.583912,21.911668,10.027885,2453.349179,1920.370438,2.191167,149.088841
min,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,11212.000000,4.800000,10465.000000,107.000000,4.000000,-11467.000000,-8348.000000,10.700000,1121.200000
25%,0.000000,14.000000,1.000000,0.000000,4.000000,4.000000,4.000000,0.000000,0.000000,0.000000,...,0.000000,15427.500000,6.800000,17209.500000,203.000000,44.000000,-1596.000000,-1212.000000,20.300000,1542.750000
50%,0.000000,16.000000,3.000000,1.000000,6.000000,6.000000,6.000000,0.000000,0.000000,0.000000,...,0.000000,16378.000000,7.000000,17974.000000,218.000000,51.000000,-14.000000,28.000000,21.800000,1637.800000
75%,1.000000,20.000000,4.000000,1.000000,8.000000,8.000000,9.000000,1.000000,1.000000,0.000000,...,0.000000,17418.500000,7.200000,18764.500000,233.000000,57.000000,1585.500000,1290.500000,23.300000,1741.850000
max,1.000000,250.000000,27.000000,1.000000,22.000000,22.000000,29.000000,2.000000,1.000000,1.000000,...,2.000000,22732.000000,8.200000,22269.000000,289.000000,92.000000,10830.000000,9333.000000,28.900000,2273.200000


### 增删特征
传统的机器学习模型大部分都是基于特征的，因此特征工程是机器学习中非常重要的一步。有时构造一个好的特征比改进一个模型带来的提升更大。这里简单展示一些特征处理的例子。首先，上面提到，特征列中有些特征信息是完全冗余的，会给模型带来不必要的计算量，可以去除。其次，相比于红蓝双方击杀、助攻的绝对值，可能双方击杀英雄的差值更能体现出当前对战的局势。因此，我们可以构造红蓝双方对应特征的差值。数据文件中已有的差值是金币差GoldDiff和经验差ExperienceDiff，实际上每个对应特征都可以构造这样的差值特征。

In [36]:
drop_freatures = ['blueGoldDiff', 'redGoldDiff', 
                 'blueExperienceDiff', 'redExperienceDiff', 
                 'blueCSPerMin', 'redCSPerMin', 
                 'blueGoldPerMin', 'redGoldPerMin']
df = data_df.drop(columns= drop_freatures)
to_delete_features = []
to_minus_features = [example[3:] for example in df.columns if example.startswith('red')]
for to_minus in to_minus_features:
    df["br" + to_minus] = df["blue"+to_minus] - df["red"+to_minus]
    to_delete_features.extend(["blue"+to_minus, "red"+to_minus])
df = df.drop(columns=to_delete_features)
df

,blueWins,brWardsPlaced,brWardsDestroyed,brFirstBlood,brKills,brDeaths,brAssists,brEliteMonsters,brDragons,brHeralds,brTowersDestroyed,brTotalGold,brAvgLevel,brTotalExperience,brTotalMinionsKilled,brTotalJungleMinionsKilled
0,0,13,-4,1,3,-3,3,0,0,0,0,643,-0.2,-8,-2,-19
1,0,0,0,-1,0,0,3,-2,-1,-1,-1,-2908,-0.2,-1173,-66,-9
2,0,0,-3,-1,-4,4,-10,1,1,0,0,-1172,-0.4,-1033,-17,18
3,0,28,-1,-1,-1,1,-5,1,0,1,0,-1321,0.0,-7,-34,8
4,0,58,2,-1,0,0,-1,-1,-1,0,0,-1004,0.0,230,-15,-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9874,1,-29,-1,1,3,-3,-2,1,1,0,0,2519,0.4,2469,-18,35
9875,1,42,-21,-1,2,-2,5,1,1,0,0,782,0.2,888,27,-8
9876,0,9,1,-1,-1,1,-6,-1,-1,0,0,-2416,-0.4,-1877,-51,-15
9877,0,-52,0,1,-1,1,2,1,1,0,0,-839,-0.6,-1085,-23,8


### 特征离散化
决策树ID3算法一般是基于离散特征的，本例中存在很多连续的数值特征，例如队伍金币。直接应用该算法每个值当作一个该特征的一个取值可能造成严重的过拟合，因此需要对特征进行离散化，即将一定范围内的值映射成一个值，例如对用户年龄特征，将0-10映射到0，11-18映射到1，19-25映射到2，25-30映射到3，等等类似，然后在决策树构建时使用映射后的值计算信息增益。

***本小节要求实现特征离散化，请补全相关代码***

In [37]:
discrete_df = df.copy() # 先复制一份数据
for c in df.columns[1:]: # 遍历每一列特征，跳过标签列
    
    if discrete_df[c].nunique() < 10: # 如果该特征取值个数小于10，则跳过离散化
        continue
    discrete_df[c] = pd.cut(discrete_df[c], bins=5)
    discrete_df[c] = discrete_df[c].cat.codes

print(discrete_df.columns[0:])
discrete_df.head()

Index(['blueWins', 'brWardsPlaced', 'brWardsDestroyed', 'brFirstBlood',
       'brKills', 'brDeaths', 'brAssists', 'brEliteMonsters', 'brDragons',
       'brHeralds', 'brTowersDestroyed', 'brTotalGold', 'brAvgLevel',
       'brTotalExperience', 'brTotalMinionsKilled',
       'brTotalJungleMinionsKilled'],
      dtype='object')


,blueWins,brWardsPlaced,brWardsDestroyed,brFirstBlood,brKills,brDeaths,brAssists,brEliteMonsters,brDragons,brHeralds,brTowersDestroyed,brTotalGold,brAvgLevel,brTotalExperience,brTotalMinionsKilled,brTotalJungleMinionsKilled
0,0,2,2,1,3,1,2,0,0,0,0,2,2,2,2,1
1,0,2,2,-1,2,2,2,-2,-1,-1,-1,1,2,2,1,2
2,0,2,2,-1,1,3,1,1,1,0,0,2,2,2,2,3
3,0,2,2,-1,2,2,2,1,0,1,0,2,2,2,1,2
4,0,3,2,-1,2,2,2,-1,-1,0,0,2,2,2,2,2


### 数据集准备
构建机器学习模型前要构建训练和测试的数据集。在本例中首先需要分开标签和特征，标签是不能作为模型的输入特征的，就好比作业和试卷答案不能在做题和考试前就告诉学生。测试一个模型在一个任务上的效果至少需要训练集和测试集，训练集用来训练模型的参数，好比学生做作业获得知识，测试集用来测试模型效果，好比期末考试考察学生学习情况。测试集的样本不应该出现在训练集中，否则会造成模型效果估计偏高，好比考试时出的题如果是作业题中出现过的，会造成考试分数不能准确衡量学生的学习情况，估计值偏高。划分训练集和测试集有多种方法，下面首先介绍的是随机取一部分如20%作测试集，剩下作训练集。sklearn提供了相关工具函数`train_test_split`。sklearn的输入输出一般为numpy的array矩阵，需要先将pandas的DataFrame取出为numpy的array矩阵。

In [38]:
all_y = discrete_df['blueWins'].values # 所有标签数据
feature_names = discrete_df.columns[1:] # 所有特征的名称
all_x = discrete_df[feature_names].values # 所有原始特征值，pandas的DataFrame.values取出为numpy的array矩阵

#处理缺失值
for i in range(all_x.shape[1]):
    if np.sum(np.isnan(all_x[:,i])): # 判断是否有缺失值
        all_x[np.where(np.isnan(all_x[:,i]))[0],i] = np.median(all_x[np.where(~np.isnan(all_x[:,i]))[0],i]) # 用中位数填充

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(all_x, all_y, test_size=0.2, random_state=RANDOM_SEED)
all_y.shape, all_x.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape # 输出数据行列信息

((9879,), (9879, 15), (7903, 15), (1976, 15), (7903,), (1976,))

###  决策树模型的实现
***本小节要求实现决策树模型，请补全算法代码***

In [39]:
class DecisionTree():
    def __init__(self, classes, features,
                 max_depth, min_samples_split,
                 impurity_t):

        self.classes = classes
        self.features = features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.impurity_t = impurity_t
        self.root = None  # 定义根节点，未训练时为空

    def impurity(self, label):
        #熵
        if self.impurity_t == 'entropy':
            unique_labels, counts = np.unique(label, return_counts=True)
            total_count = len(label)
            entropy = 0
            for count in counts:
                p = float(count) / total_count
                if p > 0:
                    entropy -= p * np.log2(p)
            return entropy
        #基尼系数
        elif self.impurity_t == 'gini':
            unique_labels, counts = np.unique(label, return_counts=True)
            gini = 1
            for count in counts:
                p = float(count) / len(label)
                gini -= p ** 2
            return gini
        #错分类
        elif self.impurity_t == 'misclassification':
            unique_labels, counts = np.unique(label, return_counts=True)
            return 1 - float(max(counts)) / len(label)

    def split_information(self, feature, feature_idx):
        unique_feature_values = np.unique(feature[:, feature_idx])
        total_count = len(feature)
        split_info = 0
        for value in unique_feature_values:
            subset_count = len(np.where(feature[:, feature_idx] == value)[0])
            p = float(subset_count) / total_count
            if p > 0:
                split_info -= p * np.log2(p)
        return split_info

    def gain_ratio(self, feature, label, feature_idx):
        gain = self.gain(feature, label, feature_idx)
        split_info = self.split_information(feature, feature_idx)
        if split_info == 0:
            return 0
        return float(gain / split_info)

    def gain(self, feature, label, feature_idx):
        base_entropy = self.impurity(label)
        unique_feature_values = np.unique(feature[:, feature_idx])
        new_entropy = 0
        for value in unique_feature_values:
            subset_idx = np.where(feature[:, feature_idx] == value)[0]
            subset_label = label[subset_idx]
            subset_entropy = self.impurity(subset_label)
            subset_prob = float(len(subset_label)) / len(label)
            new_entropy += subset_prob * subset_entropy
        return base_entropy - new_entropy

    def expand_node(self, feature, label, depth=1):
        num_samples, num_features = feature.shape
        if num_samples < self.min_samples_split or depth > self.max_depth or len(np.unique(label)) == 1:
            # 情况1：样本数太少、达到最大深度或者标签只有一类，不再分裂，返回多数类作为叶节点类别
            majority_class = self.majority_class(label)
            return {'is_leaf': True, 'class': majority_class}

        best_feature_idx = None
        best_gain_ratio = -1
        for feature_idx in range(num_features):
            current_gain_ratio = self.gain_ratio(feature, label, feature_idx)
            if current_gain_ratio > best_gain_ratio:
                best_gain_ratio = current_gain_ratio
                best_feature_idx = feature_idx

        if best_gain_ratio < 0.002:
            # 情况3：找不到有用的分裂特征，返回多数类作为叶节点类别
            majority_class = self.majority_class(label)
            return {'is_leaf': True, 'class': majority_class}

        # 情况2：找到最佳分裂特征，创建节点并递归分裂子节点
        node = {'is_leaf': False, 'feature_idx': best_feature_idx, 'children': {}}
        unique_feature_values = np.unique(feature[:, best_feature_idx])
        for value in unique_feature_values:
            subset_idx = np.where(feature[:, best_feature_idx] == value)[0]
            subset_feature = feature[subset_idx]
            subset_label = label[subset_idx]
            node['children'][value] = self.expand_node(subset_feature, subset_label, depth + 1)
        return node

    def majority_class(self, label):
        counter = Counter(label)
        return max(counter.keys(), key=lambda x: counter[x])

    def fit(self, feature, label):
        assert len(self.features) == len(feature[0])  # 输入数据的特征数目应该和模型定义时的特征数目相同
        self.root = self.expand_node(feature, label, depth=1)

    def traverse_node(self, node, feature):
        if node['is_leaf']:
            return node['class']
        feature_idx = node['feature_idx']
        feature_value = feature[feature_idx]
        if feature_value not in node['children']:
            # 处理训练集中未出现过的特征取值情况，返回当前节点下样本最多的类别
            children_classes = [self.traverse_node(child, feature) if not child['is_leaf'] else child['class']
                                for child in node['children'].values()]
            majority_class = self.majority_class(np.array(children_classes))
            return majority_class
        return self.traverse_node(node['children'][feature_value], feature)

    def predict(self, feature):
        assert len(feature.shape) == 1 or len(feature.shape) == 2  # 只能是1维或2维
        if len(feature.shape) == 1:
            return self.traverse_node(self.root, feature)
        return np.array([self.traverse_node(self.root, f) for f in feature])
    
    def export_graphviz(self, node, dot=None, parent_name=None, feature_names=None):
        if dot is None:
            dot = Digraph(comment='Decision Tree')
            dot.attr('node', shape='box')

        if node['is_leaf']:
            node_name = f"class_{node['class']}"
            dot.node(node_name, label=f"Class: {node['class']}")
            if parent_name:
                dot.edge(parent_name, node_name)
            return dot

        feature_idx = node['feature_idx']
        feature_name = feature_names[feature_idx] if feature_names else f"Feature_{feature_idx}"
        node_name = f"node_{id(node)}"
        dot.node(node_name, label=feature_name)

        if parent_name:
            dot.edge(parent_name, node_name)

        for value, child in node['children'].items():
            child_name = f"node_{id(child)}"
            dot = self.export_graphviz(child, dot, child_name, feature_names)
            edge_label = str(value)
            dot.edge(node_name, child_name, label=edge_label)

        return dot
    

        
# 定义决策树模型，传入算法参数
DT_gini = DecisionTree(classes=[0,1], features=feature_names, max_depth = 5, min_samples_split=200, impurity_t='gini')
DT_entropy = DecisionTree(classes=[0,1], features=feature_names, max_depth = 5, min_samples_split=200, impurity_t="entropy")
DT_misclassification = DecisionTree(classes=[0,1], features=feature_names, max_depth = 5, min_samples_split=200, impurity_t="misclassification")

DT_gini.fit(x_train, y_train) # 在训练集上训练
p_test = DT_gini.predict(x_test) # 在测试集上预测，获得预测值
print(p_test) # 输出预测值
print(p_test.shape)
test_acc = accuracy_score(p_test, y_test) # 将测试预测值与测试集标签对比获得准确率
print('accuracy by gini: {:.4f}'.format(test_acc)) # 输出准确率

DT_entropy.fit(x_train, y_train) # 在训练集上训练
p_test = DT_entropy.predict(x_test) # 在测试集上预测，获得预测值
print(p_test) # 输出预测值
print(p_test.shape)
test_acc = accuracy_score(p_test, y_test) # 将测试预测值与测试集标签对比获得准确率
print('accuracy by entropy: {:.4f}'.format(test_acc)) # 输出准确率

DT_misclassification.fit(x_train, y_train) # 在训练集上训练
p_test = DT_misclassification.predict(x_test) # 在测试集上预测，获得预测值
print(p_test) # 输出预测值
print(p_test.shape)
test_acc = accuracy_score(p_test, y_test) # 将测试预测值与测试集标签对比获得准确率
print('accuracy by gini: {:.4f}'.format(test_acc)) # 输出准确率

[0 1 0 ... 0 0 0]
(1976,)
accuracy by gini: 0.7151
[0 1 0 ... 0 0 0]
(1976,)
accuracy by entropy: 0.7151
[0 1 0 ... 0 0 0]
(1976,)
accuracy by gini: 0.7156


### 模型调优
第一次模型测试结果可能不够好，可以先检查调试代码是否有bug，再尝试调整参数或者优化计算方法。

### 总结
一个完整的机器学习任务包括：确定任务、数据分析、特征工程、数据集划分、模型设计、模型训练和效果测试、结果分析和调优等多个阶段，本案例以英雄联盟游戏胜负预测任务为例，给出了每个阶段的一些简单例子，帮助大家入门机器学习，希望大家有所收获！